# DrugBank

Created by: Charles Dai <br>
Credit to: Moshe Silverstein

Data Source: https://www.drugbank.ca/releases/latest

In [ ]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [ ]:
# from clustergrammer_widget import *
# net = Network(clustergrammer_widget)

In [ ]:
%load_ext autoreload
%autoreload 2

### Notebook Information

In [ ]:
print('This notebook was run on:', date.today(), '\nPython version:', sys.version)

# Initialization

In [ ]:
%%appyter code_eval

{% set group = ChoiceField(
    name='identifier',
    label='Protein Identifier Group',
    choices=['Target', 'Enzyme', 'Carrier', 'Transporter'],
    default='Target',
    section='data'
) %}

### Load Mapping Dictionaries

In [ ]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

### Output Path

In [ ]:
%%appyter code_exec

output_name = 'drugbank_' + '{{group}}'.lower()

path = 'Output/DrugBank-' + '{{group}}'
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
%%appyter hide_code
{% do SectionField(
    name='data',
    title='Load Data',
    subtitle='Upload Files from the DrugBank Protein Identifiers Datasets',
) %}

# Load Data

In [ ]:
%%appyter code_exec

df = pd.read_csv({{FileField(
    constraint='.*\.zip$',
    name='drug_identifiers', 
    label='Drug Identifiers Dataset (csv.zip)', 
    default='Input/DrugBank/drugbank_all_target_polypeptide_ids.csv.zip',
    section='data')
}}, usecols=['Gene Name', 'Drug IDs', 'Species'], index_col=0)

In [ ]:
df.head()

In [ ]:
df.shape

# Load Drug Metadata

In [ ]:
%%appyter code_exec

drug_meta = pd.read_csv({{FileField(
    constraint='.*\.zip$',
    name='drug_metadata', 
    label='External Drug Links (csv.zip)', 
    default='Input/DrugBank/drugbank_all_drug_links.csv.zip',
    section='data')
}}, usecols=['DrugBank ID', 'Name'], index_col=0)

In [ ]:
drug_meta.head()

In [ ]:
drug_meta.shape

# Pre-process Data

## Get Relevant Data

In [ ]:
# Get Relevant Species
df = df[np.logical_or.reduce([
    df['Species'] == 'Humans',
    df['Species'] == 'Mouse',
    df['Species'] == 'Rat'
])].drop('Species', axis=1)
df.head()

## Split Drug ID list

In [ ]:
df['Drug IDs'] = df['Drug IDs'].map(lambda x: x.split('; '))
df = df.explode('Drug IDs').dropna()
df.head()

In [ ]:
df.shape

## Map Drug IDs to Names

In [ ]:
df['Drug IDs'] = drug_meta.reindex(df['Drug IDs']).set_index(df.index)
df.index.name = 'Gene Symbol'
df.columns = ['Drug Name']
df.head()

# Filter Data

## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [ ]:
df = uf.map_symbols(df, symbol_lookup, remove_duplicates=True)
df.shape

# Analyze Data

## Create Binary Matrix

In [ ]:
binary_matrix = uf.binary_matrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.save_data(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

## Create Gene List

In [ ]:
gene_list = uf.gene_list(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.save_data(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

## Create Attribute List

In [ ]:
attribute_list = uf.attribute_list(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.save_data(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

## Create Gene and Attribute Set Libraries

In [ ]:
uf.save_setlib(binary_matrix, 'gene', 'up', path, output_name + '_gene_up_set')

In [ ]:
uf.save_setlib(binary_matrix, 'attribute', 'up', path, 
                           output_name + '_attribute_up_set')

## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.similarity_matrix(binary_matrix.T, 'jaccard', sparse=True)
attribute_similarity_matrix.head()

In [ ]:
uf.save_data(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

In [ ]:
# net.load_df(attribute_similarity_matrix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
# net.cluster()
# net.widget()

## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.similarity_matrix(binary_matrix, 'jaccard', sparse=True)
gene_similarity_matrix.head()

In [ ]:
uf.save_data(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene-Attribute Edge List

In [ ]:
edge_list = uf.edge_list(binary_matrix)
uf.save_data(edge_list, path, output_name + '_edge_list', 
        ext='tsv', compression='gzip')

# Create Downloadable Save File

In [ ]:
uf.archive(path)

### Link to download output files: [click here](./output_archive.zip)